In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from dotenv import load_dotenv
import os
import seaborn as sns
import pandas as pd
from twelvedata import TDClient
from datetime import datetime, timedelta

tqdm.pandas()
load_dotenv()

API_KEY = os.getenv("API_KEY")

In [2]:
INSTRUMENT_ID = "spx"
INTERVAL = "1min"
START = "2020-08-10"
END = "2023-08-11"
GET_CACHED = False
TOTAL_DAYS = (datetime.strptime(END, '%Y-%m-%d') -
              datetime.strptime(START, '%Y-%m-%d')).days
ERROR_TOLERANCE = 4


def load_data():
    print(f"Running for {TOTAL_DAYS}")

    iters = 0
    errors = 0
    current_date = START
    # Initialize an empty list to store DataFrames for each date
    all_data = []
    td = TDClient(apikey=API_KEY)

    pbar = tqdm(desc="Building Timeseries", total=TOTAL_DAYS)
    while current_date <= END:
        try:
            ts = td.time_series(
                symbol=INSTRUMENT_ID,
                interval=INTERVAL,
                timezone="Europe/Malta",
                date=current_date
            )

            ts_df = ts.as_pandas().reset_index()
            all_data.append(ts_df)
            errors = 0
        except:
            print(f"Error found, number of errors: {errors}")
            errors += 1

        if errors >= ERROR_TOLERANCE:
            print(f"Enough errors: {errors} quitting")
            break

        current_date = datetime.strptime(current_date, '%Y-%m-%d')
        current_date += timedelta(days=1)
        current_date = current_date.strftime('%Y-%m-%d')
        print(f"completed {current_date}")

        iters += 1
        pbar.update(iters)

    # Concatenate all DataFrames in the list into a single DataFrame
    combined_data = pd.concat(all_data, ignore_index=True)
    combined_data.to_json(
        f"./ts/{INSTRUMENT_ID}-{START}-{END}-{INTERVAL}.json")


if GET_CACHED:
    combined_data = pd.read_json(
        f"./ts/{INSTRUMENT_ID}-{START}-{END}-{INTERVAL}.json", orient='index'
    )
else:
    combined_data = load_data()

combined_data.describe()

Running for 1096


Building Timeseries:   0%|          | 0/1096 [00:00<?, ?it/s]

completed 2020-08-11
completed 2020-08-12
completed 2020-08-13
completed 2020-08-14
completed 2020-08-15
Error found, number of errors: 0
completed 2020-08-16
Error found, number of errors: 1
completed 2020-08-17
completed 2020-08-18
completed 2020-08-19
completed 2020-08-20
completed 2020-08-21
completed 2020-08-22
Error found, number of errors: 0
completed 2020-08-23
Error found, number of errors: 1
completed 2020-08-24
completed 2020-08-25
completed 2020-08-26
completed 2020-08-27
completed 2020-08-28
completed 2020-08-29
Error found, number of errors: 0
completed 2020-08-30
Error found, number of errors: 1
completed 2020-08-31
completed 2020-09-01
completed 2020-09-02
completed 2020-09-03
completed 2020-09-04
completed 2020-09-05
Error found, number of errors: 0
completed 2020-09-06
Error found, number of errors: 1
completed 2020-09-07
Error found, number of errors: 2
completed 2020-09-08
completed 2020-09-09
completed 2020-09-10
completed 2020-09-11
completed 2020-09-12
Error foun

AttributeError: 'NoneType' object has no attribute 'describe'